# Importing the needed liberairies

In [1]:
%matplotlib inline
# System
import os
import time
import sys, datetime

# CV
import numpy as np
import cv2
from imutils.video import FPS

# DL
import tensorflow as tf
from keras.models import Model

# Maths
from statistics import mean, median

# images and display
import imutils
from tensorflow.keras.preprocessing import image
from skimage import img_as_float
from imutils.video import VideoStream
from matplotlib import pyplot as plt

In [2]:
GREEN = (0, 255, 0)
RED = (0, 0, 255)
BLUE=(255, 0, 0)
YELLOW = (0, 255, 255)

# Loading the Pre-trained models

In [3]:
# Change the working directory
os.chdir(r"\Users\messi\OneDrive\Desktop\CIE\Computer Vision\Projects\Final-Project")

In [4]:
# Loading mask detection model
loaded_model_0 = tf.keras.models.load_model('mask_detection_pretrained')
# Loading mask status model
loaded_model_1 = tf.keras.models.load_model('mask_status_pretrained')

# Prediction

In [5]:
def predict_class(img, flag=0):
    # This function takes and image and a flag, it predicts the label of the image using
    # the appropriate model based on the value of the flag (0: mask detection, 1: mask status)
    
    # Processing the image to be suitable for prediction
    img_array = image.img_to_array(img)
    camera_float = img_as_float(img_array)
    img_batch = np.expand_dims(camera_float, axis=0)
    img_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(img_batch)
    
    # Mapping the label
    if flag ==0:
        prediction = loaded_model_0.predict(img_preprocessed)
        pred = np.argmax(prediction,axis=1)
        unique_labels = dict((v,k) for v,k in [(0,'with_mask'), (1,'without_mask')])
    else:
        prediction = loaded_model_1.predict(img_preprocessed)
        pred = np.argmax(prediction,axis=1)
        unique_labels = dict((v,k) for v,k in [(0,'Correct Mask'), (1,'Mask covers chin only'),(2,'Mask covers mouth and chin'),(3,'Mask covers mouth and nose')])
        
    pred = [unique_labels[k] for k in pred]
    return pred

In [6]:
def Start_Tracker(track_name, frame, face):
    if track_name == "KCF": tracker = cv2.TrackerKCF_create()
    elif track_name == "CSRT": tracker = cv2.TrackerCSRT_create()
    elif track_name == "MIL": tracker = cv2.TrackerMIL_create()
    tracker.init(frame, face)
    return tracker

# Real-time system

In [7]:
# Create an instance of Haar Cascades face detector
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

## Face Detection Only 

In [25]:
# Capture a video from webcam 
cap = cv2.VideoCapture(0)
#time.sleep(2.0)
# To use a video file as input 
#cap = cv2.VideoCapture(r'data\output1.mkv')
fps = FPS().start()
while True:
    # Read the frame
    _, img = cap.read()
    # Resize the frame to be of width 400
    #img = imutils.resize(img, width=400)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    color = GREEN
    for (x, y, w, h) in faces:
        # Draw rectangle around each face in the frame
        cv2.rectangle(img, (x, y - 30), (x + w, y + h + 30), color, 2)
        # Print the label on the image
    fps.update()
    fps.stop()
    info = [
        ("Status", "Detecting"),
        ("Detector", "Haar Cascades"),
        ("FPS", "{:.2f}".format(fps.fps())),
    ]
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(img, text, (10, 300 - ((i * 20) + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 1)
    cv2.imshow('img', img)
    # Stop if q key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Face detection and Tracking

### KCF Tracker

In [26]:
# Capture a video from webcam 
cap = cv2.VideoCapture(0)
counter = 0
fps = FPS().start()
while True:
    # Read the frame
    _, img = cap.read()
    # Resize the frame to be of width 400
    #img = imutils.resize(img, width=400)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    if counter == 0:
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        trackers = [Start_Tracker("KCF",img, face) for face in faces]
        counter+=1
    faces = [t.update(img)[1] for t in trackers]
    if len(faces)==0:
        counter = 0
        continue
    for (x, y, w, h) in faces:
        if (x, y, w, h) == (0, 0, 0, 0):
            counter = 0
            break
        # Draw rectangle around each face in the frame
        cv2.rectangle(img, (x, y - 30), (x + w, y + h + 30), color, 2)
        # Print the label on the image
        color = GREEN if counter <= 1 else BLUE
    fps.update()
    fps.stop()
    info = [
        ("Tracker", "KCF"),
        ("Status", "Detecting" if counter<=1 else "Tracking"),
        ("FPS", "{:.2f}".format(fps.fps())),
    ]
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(img, text, (10, 300 - ((i * 20) + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 1)
    cv2.imshow('img', img)
    # Detect every 90 frames
    if counter > 0 and counter < 90:
        counter += 1
    else:
        counter = 0
    # Stop if escape key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## CSRT Tracker

In [10]:
# Capture a video from webcam 
cap = cv2.VideoCapture(0)
counter = 0
fps = FPS().start()
while True:
    # Read the frame
    _, img = cap.read()
    # Resize the frame to be of width 400
    #img = imutils.resize(img, width=400)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    if counter == 0:
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        trackers = [Start_Tracker("CSRT",img, face) for face in faces]
        counter+=1
    faces = [t.update(img)[1] for t in trackers]
    if len(faces)==0:
        counter = 0
        continue
    for (x, y, w, h) in faces:
        if (x, y, w, h) == (0, 0, 0, 0):
            counter = 0
            break

        roi = img[y - 30:y + h + 30, x:x + w]
        # Resize the face to fit the shape of the input layer of mask detection model
        roi = cv2.resize(roi, (224, 224), interpolation=cv2.INTER_AREA)
        roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        # Run mask detector on each face
        # Draw rectangle around each face in the frame
        cv2.rectangle(img, (x, y - 30), (x + w, y + h + 30), color, 2)
        # Print the label on the image
        color = GREEN if counter <= 1 else BLUE
    fps.update()
    fps.stop()
    info = [
        ("Tracker", "CSRT"),
        ("Status", "Detecting" if counter<=1 else "Tracking"),
        ("FPS", "{:.2f}".format(fps.fps())),
    ]
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(img, text, (10, 300 - ((i * 20) + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 1)
    cv2.imshow('img', img)
    # Detect every 90 frames
    if counter > 0 and counter < 90:
        counter += 1
    else:
        counter = 0
    # Stop if q key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## MIL Tracker

In [11]:
# Capture a video from webcam 
cap = cv2.VideoCapture(0)
counter = 0
fps = FPS().start()
while True:
    # Read the frame
    _, img = cap.read()
    # Resize the frame to be of width 400
    #img = imutils.resize(img, width=400)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    if counter == 0:
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        trackers = [Start_Tracker("MIL",img, face) for face in faces]
        counter+=1
    faces = [t.update(img)[1] for t in trackers]
    if len(faces)==0:
        counter = 0
        continue
    for (x, y, w, h) in faces:
        if (x, y, w, h) == (0, 0, 0, 0):
            counter = 0
            break
        # Draw rectangle around each face in the frame
        cv2.rectangle(img, (x, y - 30), (x + w, y + h + 30), color, 2)
        # Print the label on the image
        color = GREEN if counter <= 1 else BLUE
    fps.update()
    fps.stop()
    info = [
        ("Tracker", "MIL"),
        ("Status", "Detecting" if counter<=1 else "Tracking"),
        ("FPS", "{:.2f}".format(fps.fps())),
    ]
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(img, text, (10, 300 - ((i * 20) + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 1)
    cv2.imshow('img', img)
    # Detect every 90 frames
    if counter > 0 and counter < 90:
        counter += 1
    else:
        counter = 0
    # Stop if q key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Mask Detection Model only

In [27]:
# Capture a video from webcam 
cap = cv2.VideoCapture(0)
# To use a video file as input 
#cap = cv2.VideoCapture(r'data\output1.mkv')
counter = 0
fps = FPS().start()
while True:
    # Read the frame
    _, img = cap.read()
    # Resize the frame to be of width 400
    #img = imutils.resize(img, width=400)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    if counter == 0:
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        trackers = [Start_Tracker("KCF",img, face) for face in faces]
        counter+=1
    faces = [t.update(img)[1] for t in trackers]
    if len(faces)==0:
        counter = 0
        continue
    for (x, y, w, h) in faces:
        if (x, y, w, h) == (0, 0, 0, 0):
            counter = 0
            break

        roi = img[y - 30:y + h + 30, x:x + w]
        # Resize the face to fit the shape of the input layer of mask detection model
        roi = cv2.resize(roi, (224, 224), interpolation=cv2.INTER_AREA)
        roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        # Run mask detector on each face
        label = predict_class(roi_rgb, 0)
        # Draw rectangle around each face in the frame
        cv2.rectangle(img, (x, y - 30), (x + w, y + h + 30), color, 2)
        # Print the label on the image
        if label[0]=="with_mask": color=GREEN
        else: color=RED
        cv2.putText(img, label[0], (x, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    fps.update()
    fps.stop()
    info = [
        ("Tracker", "KCF"),
        ("Status", "Detecting" if counter<=1 else "Tracking"),
        ("FPS", "{:.2f}".format(fps.fps())),
    ]
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(img, text, (10, 300 - ((i * 20) + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 1)
    cv2.imshow('img', img)
    # Stop if escape key is pressed
    if counter > 0 and counter < 90:
        counter += 1
    else:
        counter = 0
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Correctly weared mask

In [30]:
# Capture a video from webcam 
cap = cv2.VideoCapture(0)
# To use a video file as input 
#cap = cv2.VideoCapture(r'data\output3.mkv')
#Find the number of frames with mask
frames_mask = 21 * 30
results=[]
frames_per_second = cap.get(cv2.CAP_PROP_FPS)
cur_frame = 0
print(frames_per_second)
counter = 0
fps = FPS().start()
while True:
    cur_frame+=1
    threshold = 20000
    # Read the frame
    _, img = cap.read()
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    if counter == 0:
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        trackers = [Start_Tracker("KCF",img, face) for face in faces]
        counter+=1
    color = RED
    faces = [t.update(img)[1] for t in trackers]
    if len(faces)==0:
        counter = 0
        continue
    for (x, y, w, h) in faces:
        if (x, y, w, h) == (0, 0, 0, 0):
            counter = 0
            break
        if w*h <threshold:
            counter = 0
            break

        roi = img[y - 30:y + h + 30, x:x + w]
        # Resize the face to fit the shape of the input layer of mask detection model
        roi = cv2.resize(roi, (224, 224), interpolation=cv2.INTER_AREA)
        roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        # Run mask detector on each face
        label = predict_class(roi_rgb, 0)
        # if the face is classified as wearing mask, check the mask status with the second model
        if label[0] == "with_mask":
            label = predict_class(roi_rgb, 1)
            if label[0] == "Correct Mask": color = GREEN
            else: color = YELLOW
        # Draw rectangle around each face in the frame
        cv2.rectangle(img, (x, y - 30), (x + w, y + h + 30), color, 2)
        # Print the label on the image
        if label[0]=="Correct Mask": results.append(1)
        else: results.append(0)
        cv2.putText(img, label[0], (x, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    fps.update()
    fps.stop()
    info = [
        ("Tracker", "KCF"),
        ("Status", "Detecting" if counter<=1 else "Tracking"),
        ("FPS", "{:.2f}".format(fps.fps())),
    ]
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(img, text, (10, 300 - ((i * 20) + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 1)
    cv2.imshow('img', img)
    # Stop if escape key is pressed
    if counter > 0 and counter < 10:
        counter += 1
    else:
        counter = 0
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

30.0


In [23]:
cap.release()
cv2.destroyAllWindows()

In [61]:
zeros=[i for i in range(len(results)) if results[i]==0]
zeros.insert(0,0)
diff=[zeros[j+1]-zeros[j] for j in range(len(zeros)-1)]
print("Avg Successive correct frames: ",mean(diff))

Avg Successive correct frames:  139


## Prediction on a single image

In [16]:
img_path=r"C:\Users\messi\OneDrive\Desktop\CIE\Computer Vision\Projects\Final-Project\data\test8_effects.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
label=predict_class(img_array,1)
label

['Mask covers mouth and chin']